In [63]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from glob import glob
import base64
from tqdm.auto import tqdm
import fitz
import json
import math

In [3]:
load_dotenv()

True

In [4]:
client = OpenAI(
    api_key=os.environ['OPENAI_API_KEY'], 
)

In [33]:
def get_image(path: str) -> str:
    with open(path, "rb") as image_file:
        image_base64 = base64.b64encode(image_file.read()).decode('utf-8')
    
    return f"data:image/png;base64,{image_base64}"

In [34]:
def get_text(path: str) -> str:
    with open(path, "r") as txt_file:
        txt = txt_file.read()
    
    return txt

In [43]:
scale = """
1. Опыт работы:
   - 1: Менее года
   - 2: 1-2 года
   - 3: 3-5 лет
   - 4: 6-9 лет
   - 5: 10+ лет

2. Ответственность:
   - 1: Очень низкая (нельзя поручать задачи)
   - 2: Низкая (нужен сильный контроль)
   - 3: Средняя (нужно периодически проверять)
   - 4: Высокая (иногда лучше проверять)
   - 5: Очень высокая (можно положиться)

3. Креативность:
   - 1: Очень низкая (следует инструкциям)
   - 2: Низкая (вносит небольшие предложения)
   - 3: Средняя (иногда генерирует новые идеи)
   - 4: Высокая (часто предлагает инновационные решения)
   - 5: Очень высокая (постоянно предлагает инновационные решения)

4. Коммуникабельность:
   - 1: Очень низкая (затруднительное общение)
   - 2: Низкая (ограниченное общение)
   - 3: Средняя (нормальное общение)
   - 4: Высокая (хорошее общение)
   - 5: Очень высокая (исключительные навыки общения)

5. Лидерские качества:
   - 1: Очень низкие (никогда не берет на себя лидерство)
   - 2: Низкие (редко берет на себя лидерство)
   - 3: Средние (иногда берет на себя лидерство)
   - 4: Высокие (часто берет на себя лидерство)
   - 5: Очень высокие (постоянно проявляет лидерство)

6. Стрессоустойчивость:
   - 1: Очень низкая (не справляется со стрессом)
   - 2: Низкая (с трудом справляется со стрессом)
   - 3: Средняя (есть проблемы со стрессом)
   - 4: Высокая (нет проблем со стрессом)
   - 5: Очень высокая (исключительно справляется со стрессом)

7. Уровень сложности работы:
   - 1: Очень низкий (простая работа, не требующая навыков)
   - 2: Низкий (работа с базовыми требованиями)
   - 3: Средний (требует определенных навыков и знаний)
   - 4: Высокий (требует значительных знаний и навыков)
   - 5: Очень высокий (экспертный уровень, сложные задачи)

8. Средняя продолжительность работы на одном месте:
   - 1: Очень низкая (менее 6 месяцев)
   - 2: Низкая (6 месяцев - 1 год)
   - 3: Средняя (1-2 года)
   - 4: Высокая (2-4 лет)
   - 5: Очень высокая (более 4 лет)

9. Карьерный рост:
   - 1: Нет роста (остается на одном уровне)
   - 2: Низкий (незначительное повышение)
   - 3: Средний (умеренное повышение)
   - 4: Высокий (значительное повышение)
   - 5: Очень высокий (быстрое продвижение по карьерной лестнице)

10. Уровень дотошности описания:
    - 1: Очень низкий (очень мало деталей и коротко)
    - 2: Низкий (некоторые детали, но все равно коротко)
    - 3: Средний (достаточно подробное, но без лишних деталей)
    - 4: Высокий (подробное и хорошо описанное)
    - 5: Очень высокий (крайне детализированное описание)

11. Уровень заумности:
    - 1: Очень низкий (очень простые слова с ошибками)
    - 2: Низкий (простые слова без ошибок)
    - 3: Средний (обычные слова)
    - 4: Высокий (сложные слова)
    - 5: Очень высокий (очень сложные и редкие слова)

12. Уровень высокомерия:
    - 1: Очень низкий (очень скромный)
    - 2: Низкий (скромный)
    - 3: Средний (немного уверенный, но без высокомерия)
    - 4: Высокий (очень уверенный, с элементами высокомерия)
    - 5: Очень высокий (очень высокомерный)
    
13. Образование:
    - 1: Начальное, среднее или нет
    - 2: Высшее неоконченное или среднее специальное
    - 3: Высшее бакалавриат
    - 4: Высшее магистратура
    - 5: Высшее аспирантура или phd
    
14. Акадимичность:
    - 1: Только опыт работы
    - 2: Опыт работы свои проекты
    - 3: Опыт работы, свои проекты, диплом по специальности
    - 4: Свои проекты, диплом по специальности, публикации
    - 5: Диплом по специальности, публикации, преподавание
    
15. Иностранные языки:
    - 1: Нет
    - 2: Базовый английский
    - 3: Хороший английский
    - 4: Продвинутый английский или несколько языков
    - 5: Резюме на англияском
    
16. Профессиональные сертификаты:
    - 1: Нет
    - 2: Есть курс
    - 3: Есть несколько курсов
    - 4: Много курсов или расписанных сертификатов
    - 5: Много сертификатов и курсов
"""

In [99]:
def get_meme_description(image: str, txt: str) -> str: 
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"Тебе дан мем и короткое описание мема. Твоя задача - коротко описать каким был бы работником герой мема.\nОписание мема:\n{txt}\nПиши так, будто он уже является работником и ты его описываешь."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image,
                        },
                    },
                ],
            }
        ],
        max_tokens=512,
        temperature=0.0,
    )

    return response.choices[0].message.content

def get_meme_scale(image: str, txt: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"Тебе дан мем, короткое описание мема и параметры оценки со шкалой. Твоя задача - используя параметры со шкалой, описание и картинку оценить мем.\nОписание мема:\n{txt}\nПараметры со шкалой оценки:\n{scale}Ответ ДОЛЖЕН быть в формате json."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image,
                        },
                    },
                ],
            }
        ],
        max_tokens=512,
        temperature=0.0,
    )

    return response.choices[0].message.content

def get_cv_scale(txt: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"Тебе дан текст резюме и параметры оценки со шкалой. Твоя задача - используя параметры со шкалой, и текст резбме оценить это резюме.\nТекст резюме:\n{txt}\nПараметры со шкалой оценки:\n{scale}Ответ ДОЛЖЕН быть в формате json."},
                ],
            }
        ],
        max_tokens=512,
        temperature=0.0,
    )

    return response.choices[0].message.content

def get_meme_rerank(cv: str, memes: list[tuple]) -> str:
    meme1, meme2, meme3 = memes
    
    prompt = f"""
Тебе дан текст резюме и три мема, для каждого мема дано название, короткое описание и описание каким бы мем был работником. 
Твоя задача - выбрать наиболее подходящий мем.

Текст резюме:
{cv}

Мем: {meme1[0]}
Короткое описание: {meme1[1]}
Описание каким мем был бы работником: {meme1[2]}

Мем: {meme2[0]}
Короткое описание: {meme2[1]}
Описание каким мем был бы работником: {meme2[2]}

Мем: {meme3[0]}
Короткое описание: {meme3[1]}
Описание каким мем был бы работником: {meme3[2]}

Твой ответ ДОЛЖЕН содержать ТОЛЬКО название мема, больше никакого текста.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ],
        max_tokens=128,
        temperature=0.0,
    )

    return response.choices[0].message.content

In [45]:
descriptions = {}

for img_path in tqdm(glob("../data/meme/img/*")):
    image = get_image(img_path)
    image_name = img_path.split("/")[-1].split(".")[0]
    txt_path = f"../data/meme/txt/{image_name}.txt"
    txt = get_text(txt_path)
    
    # descriptions[image_name] = get_meme_description(image, txt)
    descriptions[image_name] = get_meme_scale(image, txt)

  0%|          | 0/15 [00:00<?, ?it/s]

In [46]:
for k, v in descriptions.items():
    print(f"Meme: {k}\nScale:\n{v}\n\n=========================\n\n")

Meme: жаль
Scale:
```json
{
  "опыт_работы": 2,
  "ответственность": 2,
  "креативность": 3,
  "коммуникабельность": 4,
  "лидерские_качества": 2,
  "стрессоустойчивость": 3,
  "уровень_сложности_работы": 2,
  "средняя_продолжительность_работы_на_одном_месте": 2,
  "карьерный_рост": 2,
  "уровень_дотошности_описания": 3,
  "уровень_заумности": 4,
  "уровень_высокомерия": 4,
  "образование": 3,
  "академичность": 2,
  "иностранные_языки": 2,
  "профессиональные_сертификаты": 1
}
```



Meme: ебать_ты_лох
Scale:
```json
{
  "опыт_работы": 2,
  "ответственность": 3,
  "креативность": 3,
  "коммуникабельность": 3,
  "лидерские_качества": 2,
  "стрессоустойчивость": 3,
  "уровень_сложности_работы": 2,
  "средняя_продолжительность_работы_на_одном_месте": 2,
  "карьерный_рост": 2,
  "уровень_дотошности_описания": 3,
  "уровень_заумности": 2,
  "уровень_высокомерия": 2,
  "образование": 2,
  "академичность": 1,
  "иностранные_языки": 2,
  "профессиональные_сертификаты": 1
}
```



Meme: creepy

In [98]:
def extract_text_from_pdf(pdf_path, max_pages=5):
    document = fitz.open(pdf_path)

    if len(document) > max_pages:
        raise ValueError(f"Document has {len(document)} pages, but the limit is {max_pages} pages.")
    
    text = ""

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()

    return text

In [83]:
def parse_json_scale(txt: str, return_full=False):
    splitted = txt.split('\n')
    
    if splitted[0] == '```json' and splitted[-1] == '```':
        parsed = json.loads('\n'.join(splitted[1:-1]))
        
        if return_full:
            return {k: int(v) for k, v in parsed.items()}
        else:
            try:
                return [int(v) for _, v in parsed.items()]
            except:
                raise ValueError("Invalid json format")
    else:
        raise ValueError("Invalid json format")

def match_score(meme, resume):
    return math.sqrt(sum((m - r) ** 2 for m, r in zip(meme, resume)))

In [76]:
pdf_path = glob("../data/cv/*")[0]  
text = extract_text_from_pdf(pdf_path)
cv_scale = get_cv_scale(text)

In [77]:
pdf_path

'../data/cv/Edwin_Paco_Software_Developer.pdf'

In [78]:
cv_scale_parsed = parse_json_scale(cv_scale)

In [79]:
descriptions_parsed = {k: parse_json_scale(v) for k, v in descriptions.items()}

In [80]:
best_match = min(
    descriptions_parsed, 
    key=lambda meme: match_score(descriptions_parsed[meme], cv_scale_parsed))

print(f"Наиболее подходящий мем: {best_match}")

Наиболее подходящий мем: пух_смокинге_шаблон


In [81]:
scored_memes = [
    (meme, match_score(scores, cv_scale_parsed)) 
    for meme, scores in descriptions_parsed.items()]

sorted_memes = sorted(scored_memes, key=lambda x: x[1])

for meme, score in sorted_memes:
    print(f"Мем: {meme}, Скор: {score}")

Мем: пух_смокинге_шаблон, Скор: 3.3166247903554
Мем: сверхразум, Скор: 3.7416573867739413
Мем: da_poebat_mne_gosling, Скор: 4.358898943540674
Мем: creepy_wonka, Скор: 4.58257569495584
Мем: futurama_fry, Скор: 5.196152422706632
Мем: chalamet, Скор: 5.477225575051661
Мем: большой_доге, Скор: 6.855654600401044
Мем: жаль, Скор: 6.928203230275509
Мем: Гигачад, Скор: 7.0710678118654755
Мем: пук_среньк, Скор: 7.0710678118654755
Мем: ебать_ты_лох, Скор: 7.211102550927978
Мем: y_u_no_meme, Скор: 7.745966692414834
Мем: troll_face, Скор: 7.874007874011811
Мем: lyagushka_plachet, Скор: 8.660254037844387
Мем: forever_alone, Скор: 8.94427190999916


In [84]:
descriptions_json = {k: parse_json_scale(v, return_full=True) for k, v in descriptions.items()}

In [87]:
with open("../data/meme/scores.json", "w", encoding="utf-8") as json_file:
    json.dump(descriptions_json, json_file, ensure_ascii=False, indent=4)

In [88]:
with open("../data/meme/scores.json", "r", encoding="utf-8") as json_file:
    loaded_data = json.load(json_file)

In [93]:
descriptions_text = {}

for img_path in tqdm(glob("../data/meme/img/*")):
    image = get_image(img_path)
    image_name = img_path.split("/")[-1].split(".")[0]
    txt_path = f"../data/meme/txt/{image_name}.txt"
    txt = get_text(txt_path)

    # descriptions[image_name] = get_meme_description(image, txt)
    descriptions_text[image_name] = get_meme_description(image, txt)

  0%|          | 0/15 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [95]:
with open("../data/meme/descriptions.json", "w", encoding="utf-8") as json_file:
    json.dump(descriptions_text, json_file, ensure_ascii=False, indent=4)

In [96]:
with open("../data/meme/descriptions.json", "r", encoding="utf-8") as json_file:
    loaded_data = json.load(json_file)

In [97]:
loaded_data

{'жаль': 'Герой мема является работником, который постоянно стремится продемонстрировать свои знания и умения, используя сложные термины и избыточные объяснения. Он часто говорит много, но при этом его идеи могут быть не всегда понятны коллегам. Несмотря на его уверенность в собственном интеллекте, его подход иногда вызывает недоумение и даже раздражение у команды. Он может быть полезен в ситуациях, требующих глубокого анализа, но его манера общения затрудняет взаимодействие и совместную работу.',
 'ебать_ты_лох': 'Герой мема работает в офисе, но его подход к делу вызывает удивление у коллег. Он часто опаздывает, забывает о важных встречах и не справляется с простыми задачами. Несмотря на это, он всегда находит способ привлечь внимание к себе, рассказывая забавные истории о своих неудачах. Коллеги с недоумением наблюдают за его попытками справиться с работой, и иногда даже смеются над его «оригинальными» решениями. В целом, он — яркий пример того, как не стоит работать, но при этом ост